In [ ]:
num_threads = Threads.nthreads()

In [2]:
using Distributions
using GeometricIntegrators
using Optim
using Random
using Flux
using Enzyme
using Zygote
using Distances
using Symbolics
using Plots
using RuntimeGeneratedFunctions
RuntimeGeneratedFunctions.init(@__MODULE__)

gr()

_prod(a, b, c, arrs...) = a .* _prod(b, c, arrs...)
_prod(a, b) = a .* b
_prod(a) = a

# generates a vector out of symbolic arrays (p,q) with a certain dimension
function get_z_vector(dims)
    @variables q[1:dims]
    @variables p[1:dims]
    z = vcat(q,p)
    return z
end

# make combinations of bases of just the order that is given 
# e.g order = 2 will give just the bases whose powers sum to 2
function poly_combos(z, order, inds...)
    if order == 0
        return Num[1]
    elseif order == length(inds)
        return [_prod([z[i] for i in inds]...)]
    else
        start_ind = length(inds) == 0 ? 1 : inds[end]
        return vcat([poly_combos(z, order, inds..., j) for j in start_ind:length(z)]...)
    end
end

# gives all bases monomials up to a certain order
function primal_monomial_basis(z, order::Int)
    return Vector{Symbolics.Num}(vcat([poly_combos(z, i) for i in 1:order]...))
end

# calculates coefficient bases up to a certain order
# mostly for use with trigonometric functions example sin(k*z),
# where k is the coefficient
function primal_coeff_basis(z, max_coeff::Int)
    return Vector{Symbolics.Num}(vcat([k .* z for k in 1:max_coeff]...))
end

# calculates +,-,*,/ between states as a new basis
# the return output is a set to avoid duplicates
function primal_operator_basis(z, operator)
    return Vector{Symbolics.Num}([operator(z[i], z[j]) for i in 1:length(z)-1 for j in i+1:length(z)] ∪ [operator(z[j], z[i]) for i in 1:length(z)-1 for j in i+1:length(z)])
end

function primal_power_basis(z, max_power::Int)
    if max_power > 0
        return Vector{Symbolics.Num}(vcat([z.^i for i in 1:max_power]...))
    elseif max_power < 0
        return Vector{Symbolics.Num}(vcat([z.^-i for i in 1:abs(max_power)]...))
    end
end

function polynomial_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
    primes = primal_monomial_basis(z, polyorder)
    primes = vcat(primes, primal_coeff_basis(z, max_coeff))
    if operator !== nothing
        primes = vcat(primes, primal_operator_basis(z, operator))
    end
    return primes
end

# function trigonometric_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
#     # only on the q's
#     primes = polynomial_basis(z[1:Int(end/2)], polyorder = polyorder, operator = operator, max_coeff = max_coeff)
#     # primes = polynomial_basis(z, polyorder = polyorder, operator = operator, max_coeff = max_coeff)
#     return vcat(sin.(primes), cos.(primes))
# end
function trigonometric_basis(z::Vector{Symbolics.Num} = get_z_vector(2), coeffs = nothing, polyorder::Int = 0, operator = nothing, max_coeff::Int = 0)
    # primes = polynomial_basis(z[1:Int(end/2)], polyorder = polyorder, operator = operator, max_coeff = max_coeff)
    primes = polynomial_basis(z, polyorder = polyorder, operator = operator, max_coeff = max_coeff)
    if coeffs != nothing
    return vcat(sin.(collect(coeffs[1:Int(end/2)] .* primes)), cos.(collect(coeffs[Int(end/2)+1:end] .* primes)))
        # return cos.(collect(coeffs .* primes))
    else 
        return vcat(sin.(primes), cos.(primes))
        # return cos.(primes)
    end
end

function exponential_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
    primes = polynomial_basis(z, polyorder = polyorder, operator = operator, max_coeff = max_coeff)
    return exp.(primes)
end

function logarithmic_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
    primes = polynomial_basis(z, polyorder = polyorder, operator = operator, max_coeff = max_coeff)
    return log.(abs.(primes))
end

function mixed_states_basis(basis::Vector{Symbolics.Num}...)
    mixed_states = Tuple(basis)
    
    ham = Vector{Symbolics.Num}()
    for i in eachindex(mixed_states)
        for j in i+1:lastindex(mixed_states)
            ham = vcat(ham, [mixed_states[i][k] * mixed_states[j][l] for k in 1:length(mixed_states[i]) for l in 1:length(mixed_states[j])])
        end
    end
    
    return Vector{Symbolics.Num}(ham)
end

# returns the number of required coefficients for the basis
function get_numCoeffs(basis::Vector{Symbolics.Num})
    return length(basis)
end

#TODO: maybe basis shouldn't be variable number of arguments
# gets a vector of combinations of hamiltonian basis
function get_basis_set(basis::Vector{Symbolics.Num}...)
    # gets a vector of combinations of basis
    basis = vcat(basis...)
    
    # removes duplicates
    basis = Vector{Symbolics.Num}(collect(unique(basis)))

    return basis
end

#TODO: maybe basis shouldn't be a variable number of arguments
# returns a function that can build the gradient of the hamiltonian
function ΔH_func_builder(d::Int, z::Vector{Symbolics.Num} = get_z_vector(d), basis::Vector{Symbolics.Num}...) 
    # nd is the total number of dimensions of all the states, e.g. if q,p each of 3 dims, that is 6 dims in total
    nd = 2d
    Dz = Differential.(z)
    
    # collects and sums combinations of basis and coefficients
    #TODO: maybe not needed because the basis is initially made by calling get_basis_set
    # basis = get_basis_set(basis...)
   
    # gets number of terms in the basis
    # @variables a[1:get_numCoeffs(basis)]
    @variables a[1:length(basis...)+(2*28)] #add a variable for each coefficient of the trig basis and also for each trig argument

    # test code for trig argument bases
    trig_basis = trigonometric_basis(z, a[1:28], 2)
    poly_trig_basis = polynomial_basis(trig_basis, polyorder = 1)

    basis = get_basis_set(basis..., poly_trig_basis)
    
    # collect and sum combinations of basis and coefficients
    ham = sum(collect(a[29:end] .* basis)) #start count of (a) coefficients from after the number of trig arguments
    # ham = sum(collect(a .* basis))
    
    # gives derivative of the hamiltonian, but not the skew-symmetric true one
    f = [expand_derivatives(dz(ham)) for dz in Dz]

    #simplify the expression potentially to make it faster
    f = simplify(f)
    
    # line below makes the vector into a hamiltonian vector field by multiplying with the skew-symmetric matrix
    ∇H = vcat(f[d+1:2d], -f[1:d])
    
    # builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
    ∇H_eval = @RuntimeGeneratedFunction(Symbolics.inject_registered_module_functions(build_function(∇H, z, a)[2]))
    
    return ∇H_eval
end

struct HamiltonianSINDy{T, GHT}
    basis::Vector{Symbolics.Num} # the augmented basis for sparsification
    analytical_fθ::GHT
    z::Vector{Symbolics.Num} 
    λ::T # Sparsification Parameter
    noise_level::T # Noise amplitude added to the data
    noiseGen_timeStep::T # Time step for the integrator to get noisy data 
    nloops::Int # Sparsification Loops
    batch_size::Int # Batch size for training
    basis_coeff::Float64 # Coefficient for the coefficients of the basis
    
    function HamiltonianSINDy(basis::Vector{Symbolics.Num},
        analytical_fθ::GHT = missing,
        z::Vector{Symbolics.Num} = get_z_vector(2);
        λ::T = 0.05,
        noise_level::T = 0.00,
        noiseGen_timeStep::T = 0.05,
        nloops::Int = 10,
        batch_size::Int = 1,
        basis_coeff::Float64) where {T, GHT <: Union{Base.Callable,Missing}}

        new{T, GHT}(basis, analytical_fθ, z, λ, noise_level, noiseGen_timeStep, nloops, batch_size::Int, basis_coeff::Float64)
    end
end

function gen_noisy_ref_data(method::HamiltonianSINDy, x)
    # initialize timestep data for analytical solution
    tstep = method.noiseGen_timeStep
    tspan = (zero(tstep), tstep)

    function next_timestep(x)
        prob_ref = ODEProblem((dx, t, x, params) -> method.analytical_fθ(dx, x, params, t), tspan, tstep, x)
        sol = integrate(prob_ref, Gauss(2))
        sol.q[end]
    end

    data_ref = [next_timestep(_x) for _x in x]

    # add noise
    data_ref_noisy = [_x .+ method.noise_level .* randn(size(_x)) for _x in data_ref]

    return data_ref_noisy

end

struct TrainingData{AT<:AbstractArray}
    x::AT # initial condition
    ẋ::AT # initial condition
    y::AT # noisy data at next time step

    TrainingData(x::AT, ẋ::AT, y::AT) where {AT} = new{AT}(x, ẋ, y)
    TrainingData(x::AT, ẋ::AT) where {AT} = new{AT}(x, ẋ)
end

In [ ]:
# --------------------
# Setup
# --------------------

println("Setting up...")

# 2D system with 4 variables [q₁, q₂, p₁, p₂]
const nd = 4

z = get_z_vector(Int(nd/2))
polynomial = polynomial_basis(z, polyorder=3)

# trigonometric  = trigonometric_basis(z, max_coeff=1)

# prime_diff = primal_operator_basis(z, -)
# basis = get_basis_set(polynomial, trigonometric, prime_diff)

# basis = get_basis_set(polynomial, trigonometric)

basis = get_basis_set(polynomial)
# test_basis = vcat(z[3]^2, z[4]^2)
basis = get_basis_set(basis)
# initialize analytical function, keep λ smaller than ϵ so system is identifiable
ϵ = 0.5
m = 1

# two-dim simple harmonic oscillator (not used anywhere only in case some testing needed)
# H_ana(x, p, t) = ϵ * x[1]^2 + ϵ * x[2]^2 + 1/(2*m) * x[3]^2 + 1/(2*m) * x[4]^2
# H_ana(x, p, t) = cos(x[1]) + cos(x[2]) + 1/(2*m) * x[3]^2 + 1/(2*m) * x[4]^2

# Gradient function of the 2D hamiltonian
# grad_H_ana(x) = [x[3]; x[4]; -2ϵ * x[1]; -2ϵ * x[2]]
grad_H_ana(x) = [x[3]; x[4]; sin(x[1]); sin(x[2])]
function grad_H_ana!(dx, x, p, t)
    dx .= grad_H_ana(x)
end

# ------------------------------------------------------------
# Training Data
# ------------------------------------------------------------

println("Generate Training Data...")

# number of samples
num_samp = 10

# samples in p and q space
samp_range = LinRange(-5, 5, num_samp)

# initialize vector of matrices to store ODE solve output

# s depend on size of nd (total dims), 4 in the case here so we use samp_range x samp_range x samp_range x samp_range
s = collect(Iterators.product(fill(samp_range, nd)...))


# compute vector field from x state values
x = [collect(s[i]) for i in eachindex(s)]

dx = zeros(nd)
p = 0
t = 0
ẋ = [grad_H_ana!(copy(dx), _x, p, t) for _x in x]


# ----------------------------------------
# Compute Sparse Regression
# ----------------------------------------

# choose SINDy method
# (λ parameter must be close to noise value so that only coeffs with value around the noise are sparsified away)
# noiseGen_timeStep chosen randomly for now
method = HamiltonianSINDy(basis, grad_H_ana!, z, λ = 0.05, noise_level = 0.0, noiseGen_timeStep = 0.0, batch_size = 256, basis_coeff = 0.62)

# generate noisy references data at next time step
# y = gen_noisy_ref_data(method, x)

# Change to matrices for faster computations
x = hcat(x...)
ẋ = hcat(ẋ...)
# y = hcat(y...)

# collect training data
tdata = TrainingData(x, ẋ)

In [ ]:
# dimension of system
d = size(tdata.x)[1] ÷ 2

# returns function that builds hamiltonian gradient through symbolics
fθ = ΔH_func_builder(d, method.z, method.basis)

In [ ]:
# # test code for trig argument bases
# @variables a[1:42]
# tri_basis = trigonometric_basis(get_z_vector(2), polyorder=1)
# poly_trig_basis = polynomial_basis(tri_basis, polyorder = 1)

# basis = get_basis_set(basis, poly_trig_basis)

@variables a[1:90]
# test code for trig argument bases
trig_basis = trigonometric_basis(z, a[1:28], 2)
poly_trig_basis = polynomial_basis(trig_basis, polyorder = 1)
# test_basis = vcat(z[3]^2, z[4]^2)
basis = get_basis_set(basis, poly_trig_basis)
# poly_trig_basis
# # collect and sum combinations of basis and coefficients
# ham = (collect(a[9:end] .* basis))

method = HamiltonianSINDy(basis, grad_H_ana!, z, λ = 0.05, noise_level = 0.0, noiseGen_timeStep = 0.0, batch_size = 256, basis_coeff = 0.62)

In [ ]:
function set_model(data, basis)
    #TODO: updated ld to be a parameter
    ld = size(data.x)[1]
    ndim = size(data.x)[1]

    encoder = Chain(
    Dense(4 => 4)
    # Dense(ndim => ld, sigmoid), 
    # Dense(ld => ld)
    )

    decoder = Chain(
    Dense(4 => 4)
    # Dense(ld => ndim, sigmoid),  
    # Dense(ndim => ndim)
    )

    model = ( 
        (W = encoder,),
        (W = decoder,),
        (W = ones(Float32, get_numCoeffs(basis)+28),), #add the number of trig arguments coefficients to the number of coefficients
        # (W = zeros(Float32, get_numCoeffs(method.basis)+28), ), #add the number of trig arguments coefficients to the number of coefficients
    )
    return model
end

# ------Enzyme Code------

In [ ]:
function batched_jacobian(model_layer, x_batch)
    output_dim = size(model_layer(x_batch[:, 1]))[1]
    batch_size = size(x_batch, 2)
    
    batch_jac = zeros(output_dim, batch_size, size(x_batch, 1))
    
    for i in 1:batch_size
        x_input = x_batch[:, i]
        jac = Flux.jacobian(model_layer, x_input)[1]
        batch_jac[:, i, :] = jac
    end
    return batch_jac
end

# Get ż from dz/dx and ẋ
function enc_ż(enc_jac_batch, ẋ_batch)
    ż_ref = zero(ẋ_batch)
    for i in 1:size(enc_jac_batch, 2)
        ż_ref[:, i] = enc_jac_batch[:,i,:] * (ẋ_batch[:,i])
    end
    return ż_ref
end

function evaluate_fθ(fθ, enc_x_batch, coeffs)
    f = zero(enc_x_batch[:,1])
    out = zero(enc_x_batch)
    for i in 1:size(enc_x_batch, 2)
        fθ(f, enc_x_batch[:,i], coeffs)
        out[:,i] = f
    end
    return out
end

# Get ẋ from decoder derivative (dx/dz) and ż
function dec_ẋ(dec_jac_batch, ż)
    dec_mult_ẋ = zero(ż)
    for i in 1:size(dec_jac_batch, 2)
        dec_mult_ẋ[:, i] = dec_jac_batch[:,i,:] * ż[:,i]
    end
    return dec_mult_ẋ
end

function Diff_ż(grad_fθ, ż_ref)
    return sum(abs2, (grad_fθ - ż_ref))
end

function Diff_ẋ(dec_jac_batch, grad_fθ, ẋ_ref)
    ẋ_SINDy = zero(ẋ_ref)
    for i in 1:size(dec_jac_batch, 2)
        ẋ_SINDy[:, i] = dec_jac_batch[:,i,:] * grad_fθ[:,i]
    end
    return sum(abs2, (ẋ_SINDy - ẋ_ref))
end

function set_coeffs(model_Coeffs, Ξ, biginds)
    coeffs = zero(model_Coeffs)
    coeffs[biginds] = Ξ
    return coeffs
end

function loss(model, x_batch, ẋ_batch, ż_ref_batch, dec_jac_batch, alphas, method)
    enc_x_batch = model[1].W(x_batch)

    coeffs = model[3].W

    # Compute the reconstruction loss for the entire batch
    L_r = sum(abs2, model[2].W(enc_x_batch) - x_batch)

    # Note: grad_fθ, dec_mult_ẋ, and L_c in loss function so model acts on terms in loss function
    # and gradient can see that and use that for its update calculations

    # encoded gradient from SINDy
    grad_fθ = evaluate_fθ(fθ, enc_x_batch, coeffs)

    # Difference b/w encoded gradients from SINDy and reference
    L_ż = alphas / 100 * Diff_ż(grad_fθ, ż_ref_batch)

    # Difference b/w decoded-encoded gradients from SINDy against reference
    ẋ_diff = Diff_ẋ(dec_jac_batch, grad_fθ, ẋ_batch)
    L_ẋ = alphas * ẋ_diff

    # Compute the total loss for the entire batch
    batchLoss = L_r + L_ż + L_ẋ

    # Mean of the coefficients averaged
    L_c = sum(abs, coeffs) / length(coeffs)

    batch_loss_average = batchLoss / size(x_batch, 2) + method.basis_coeff * L_c
    
    return batch_loss_average
end

In [ ]:
model[1].W.layers[1].bias .= 0
model[2].W.layers[1].bias .= 0

In [ ]:
using LinearAlgebra
model = set_model(tdata, method.basis)
model[1].W.layers[1].weight .= Matrix(LinearAlgebra.I, 4, 4)
model[2].W.layers[1].weight .= Matrix(LinearAlgebra.I, 4, 4)

In [ ]:
# Define initial model
# model = set_model(tdata, method)

# Flux gradient has problem working with the structure data directly
x = Float32.(tdata.x)
ẋ = Float32.(tdata.ẋ)

total_samples = size(x)[2]
num_batches = ceil(Int, total_samples / method.batch_size)

# Coefficients for the loss_kernel terms
alphas = round(sum(abs2, x) / sum(abs2, ẋ), sigdigits = 3)

# Derivatives of the encoder and decoder
enc_jac = batched_jacobian(model[1].W, x)
dec_jac = batched_jacobian(model[2].W, model[1].W(x))

# encoded gradient ż = dz/dx*ẋ
ż_ref = enc_ż(enc_jac, ẋ)

# initial guess
println("Initial Guess...")

opt = Adam(0.001, (0.9, 0.8))

# Set up the optimizer's state
opt_state = Flux.setup(opt, model)

println("Coeffs: $(model[3].W)")
println()
println("initial total loss: ", loss(model, x, ẋ, ż_ref, dec_jac, alphas, method))

# Array to store the losses
Initial_loss_array = Vector{Float32}()

Enzyme loss only falls when model_gradients is inside the epochs loop

Reason for the warning BLAS fallback warning below can be found in the comment here: https://discourse.julialang.org/t/warning-linking-two-modules-of-different-target-triples-bcloader-start/93864/3

In [ ]:
for epoch in 1:500
    model_gradients = deepcopy(model)
    epoch_loss = 0.0
    # Shuffle the data indices for each epoch
    shuffled_indices = shuffle(1:total_samples)
    for batch in 1:num_batches
        # Get the indices for the current batch
        batch_start = (batch - 1) * method.batch_size + 1
        batch_end = min(batch * method.batch_size, total_samples)
        batch_indices = shuffled_indices[batch_start:batch_end]

        # Extract the data for the current batch
        x_batch = x[:, batch_indices]
        ẋ_batch = ẋ[:, batch_indices]

        # Derivatives of the encoder and decoder
        enc_jac_batch = batched_jacobian(model[1].W, x_batch)
        dec_jac_batch = batched_jacobian(model[2].W, model[1].W(x_batch))
        
        # Get the encoded derivative: ż
        ż_ref_batch = enc_ż(enc_jac_batch, ẋ_batch)
        
        # Compute gradients using Enzyme
        Enzyme.autodiff(Reverse, (model, x_batch, ẋ_batch, ż_ref_batch, dec_jac_batch, alphas, method) -> loss(model, x_batch, ẋ_batch, ż_ref_batch, dec_jac_batch, alphas, method), Active, Duplicated(model, model_gradients), Const(x_batch), Const(ẋ_batch), Const(ż_ref_batch), Const(dec_jac_batch), Const(alphas), Const(method))

        # Update the parameters
        Flux.Optimise.update!(opt_state, model, model_gradients)

        # Accumulate the loss for the current batch
        epoch_loss += loss(model, x_batch, ẋ_batch, ż_ref_batch, dec_jac_batch, alphas, method)
    end
    # Compute the average loss for the epoch
    epoch_loss /= num_batches

    # Store the epoch loss
    push!(Initial_loss_array, epoch_loss)

    # Print loss after some iterations
    if epoch % 100 == 0
        println("Epoch $epoch: Average Loss: $epoch_loss")
        println("Epoch $epoch: Coefficients: $(model[3].W)")
        println()
    end
end 

# Plot the initial loss array
display(plot(log.(Initial_loss_array), label = "Initial Optimization Loss"))

have to have cos(p), sin(p) basis as well because the matrix mixes bases and cos(q) basis ends up depending on trig (p) bases as well in the encoded space

In [ ]:
# Array to store the losses of each SINDy loop
SINDy_loss_array = Vector{Vector{Float32}}()  # Store vectors of losses

# Initialize smallinds before the loop
smallinds = falses(size(model[3].W))

for n in 1:method.nloops
    println("Iteration #$n...")
    println()
    # find coefficients below λ threshold
    smallinds .= abs.(model[3].W) .< method.λ

    biginds = .~smallinds

    # check if there are any small coefficients != 0 left
    all(model[3].W[smallinds] .== 0) && break

    # set all small coefficients to zero
    model[3].W[smallinds] .= 0

    # Set up the optimizer's state
    Ξ = model[3].W[biginds]

    function sparse_loss(enc_paras, dec_paras, Ξ, model_Coeffs, x_batch, ẋ_batch, ż_ref, dec_jac_batch, alphas, method, biginds)
        coeffs = set_coeffs(model_Coeffs, Ξ, biginds)

        enc_x_batch = enc_paras(x_batch)
    
        # Compute the reconstruction loss for the entire batch
        L_r = sum(abs2, dec_paras(enc_x_batch) - x_batch)
    
        # Note: grad_fθ, dec_mult_ẋ, and L_c in loss function so model acts on terms in loss function
        # and gradient can see that and use that for its update calculations
    
        # encoded gradient from SINDy
        grad_fθ = evaluate_fθ(fθ, enc_x_batch, coeffs)
    
        # Difference b/w encoded gradients from SINDy and reference
        L_ż = alphas / 10 * Diff_ż(grad_fθ, ż_ref)
    
        # Difference b/w decoded-encoded gradients from SINDy against reference
        ẋ_diff = Diff_ẋ(dec_jac_batch, grad_fθ, ẋ_batch)
        L_ẋ = alphas * ẋ_diff
    
        # Compute the total loss for the entire batch
        batchLoss = L_r + L_ż + L_ẋ
    
        # Mean of the coefficients averaged
        L_c = sum(abs, model_Coeffs) / length(model_Coeffs)
    
        batch_loss_average = batchLoss / size(x_batch, 2) + method.basis_coeff * L_c
        
        return batch_loss_average
    end
    
    # Flux gradient has problem working with the structure data directly
    x = Float32.(tdata.x)
    ẋ = Float32.(tdata.ẋ)

    total_samples = size(x)[2]
    num_batches = ceil(Int, total_samples / method.batch_size)

    # Coefficients for the loss_kernel terms
    alphas = round(sum(abs2, x) / sum(abs2, ẋ), sigdigits = 3)

    # Array to store the losses of each epoch
    epoch_loss_array = Vector{Float64}()

    # Set up the optimizer's state
    opt_state = Flux.setup(Adam(), (model[1].W, model[2].W, Ξ))

    for epoch in 1:500
        # Gradients of the encoder, decoder and model coefficients
        grad_W1 = deepcopy(model[1].W)
        grad_W2 = deepcopy(model[2].W)
        grad_W3 = deepcopy(Ξ)
        
        epoch_loss = 0.0
        # Shuffle the data indices for each epoch
        shuffled_indices = shuffle(1:total_samples)

        for batch in 1:num_batches
            # Get the indices for the current batch
            batch_start = (batch - 1) * method.batch_size + 1
            batch_end = min(batch * method.batch_size, total_samples)
            batch_indices = shuffled_indices[batch_start:batch_end]

            # Extract the data for the current batch
            x_batch = x[:, batch_indices]
            ẋ_batch = ẋ[:, batch_indices]

            # Derivatives of the encoder and decoder
            enc_jac_batch = batched_jacobian(model[1].W, x_batch)
            dec_jac_batch = batched_jacobian(model[2].W, model[1].W(x_batch))
            
            # Get the encoded derivative: ż
            ż_ref_batch = enc_ż(enc_jac_batch, ẋ_batch)

            # Compute gradients using Enzyme
            Enzyme.autodiff(Reverse, (enc_paras, dec_paras, Ξ, model_Coeffs, x_batch, ẋ_batch, ż_ref_batch, dec_jac_batch, alphas, method, biginds) -> sparse_loss(enc_paras, dec_paras, Ξ, model_Coeffs, x_batch, ẋ_batch, ż_ref_batch, dec_jac_batch, alphas, method, biginds), Active, Duplicated(model[1].W, grad_W1), Duplicated(model[2].W, grad_W2), Duplicated(Ξ, grad_W3), Const(model[3].W), Const(x_batch), Const(ẋ_batch), Const(ż_ref_batch), Const(dec_jac_batch), Const(alphas), Const(method), Const(biginds))

            # Update the parameters
            Flux.Optimise.update!(opt_state, (model[1].W, model[2].W, Ξ), (grad_W1, grad_W2, grad_W3))

            # Accumulate the loss for the current batch
            epoch_loss += sparse_loss(model[1].W, model[2].W, Ξ, model[3].W, x_batch, ẋ_batch, ż_ref_batch, dec_jac_batch, alphas, method, biginds)
        end

        model[3].W[biginds] .= Ξ

        # Compute the average loss for the epoch
        epoch_loss /= num_batches
        
        # Store the epoch loss
        push!(epoch_loss_array, epoch_loss)

        # Print loss after some iterations
        if epoch % 100 == 0
            println("Epoch $epoch: Loss: $epoch_loss")
            println("Coefficients: $(model[3].W))")
            println()
        end
    end

    # Store the SINDy loop loss
    push!(SINDy_loss_array, epoch_loss_array)
end

# Convert vector of vectors to a single vector
SINDy_loss_array = vcat(SINDy_loss_array...)

# Plot the initial loss array
display(plot(log.(SINDy_loss_array), label = "SINDy Optimization Loss"))

In [ ]:
# Array to store the losses of each epoch
final_loss_array = Vector{Float64}()

# find coefficients below λ threshold
smallinds .= abs.(model[3].W) .<= 0

biginds = .~smallinds

# Set up the optimizer's state
Ξ = model[3].W[biginds]

function sparse_loss(enc_paras, dec_paras, Ξ, model_Coeffs, x_batch, ẋ_batch, ż_ref, dec_jac_batch, alphas, method, biginds)
    coeffs = set_coeffs(model_Coeffs, Ξ, biginds)

    enc_x_batch = enc_paras(x_batch)

    # Compute the reconstruction loss for the entire batch
    L_r = sum(abs2, dec_paras(enc_x_batch) - x_batch)

    # Note: grad_fθ, dec_mult_ẋ, and L_c in loss function so model acts on terms in loss function
    # and gradient can see that and use that for its update calculations

    # encoded gradient from SINDy
    grad_fθ = evaluate_fθ(fθ, enc_x_batch, coeffs)

    # Difference b/w encoded gradients from SINDy and reference
    L_ż = alphas / 10 * Diff_ż(grad_fθ, ż_ref)

    # Difference b/w decoded-encoded gradients from SINDy against reference
    ẋ_diff = Diff_ẋ(dec_jac_batch, grad_fθ, ẋ_batch)
    L_ẋ = alphas * ẋ_diff

    # Compute the total loss for the entire batch
    batchLoss = L_r + L_ż + L_ẋ

    # Mean of the coefficients averaged
    L_c = sum(abs, model_Coeffs) / length(model_Coeffs)

    batch_loss_average = batchLoss / size(x_batch, 2) + method.basis_coeff * L_c
    
    return batch_loss_average
end

for epoch in 1:500
    # Gradients of the encoder, decoder and model coefficients
    grad_W1 = deepcopy(model[1].W)
    grad_W2 = deepcopy(model[2].W)
    grad_W3 = deepcopy(Ξ)
    
    epoch_loss = 0.0
    # Shuffle the data indices for each epoch
    shuffled_indices = shuffle(1:total_samples)

    for batch in 1:num_batches
        # Get the indices for the current batch
        batch_start = (batch - 1) * method.batch_size + 1
        batch_end = min(batch * method.batch_size, total_samples)
        batch_indices = shuffled_indices[batch_start:batch_end]

        # Extract the data for the current batch
        x_batch = x[:, batch_indices]
        ẋ_batch = ẋ[:, batch_indices]

        # Derivatives of the encoder and decoder
        enc_jac_batch = batched_jacobian(model[1].W, x_batch)
        dec_jac_batch = batched_jacobian(model[2].W, model[1].W(x_batch))
        
        # Get the encoded derivative: ż
        ż_ref_batch = enc_ż(enc_jac_batch, ẋ_batch)

        # Compute gradients using Enzyme
        Enzyme.autodiff(Reverse, (enc_paras, dec_paras, Ξ, model_Coeffs, x_batch, ẋ_batch, ż_ref_batch, dec_jac_batch, alphas, method, biginds) -> sparse_loss(enc_paras, dec_paras, Ξ, model_Coeffs, x_batch, ẋ_batch, ż_ref_batch, dec_jac_batch, alphas, method, biginds), Active, Duplicated(model[1].W, grad_W1), Duplicated(model[2].W, grad_W2), Duplicated(Ξ, grad_W3), Const(model[3].W), Const(x_batch), Const(ẋ_batch), Const(ż_ref_batch), Const(dec_jac_batch), Const(alphas), Const(method), Const(biginds))

        # Update the parameters
        Flux.Optimise.update!(opt_state, (model[1].W, model[2].W, Ξ), (grad_W1, grad_W2, grad_W3))

        # Accumulate the loss for the current batch
        epoch_loss += sparse_loss(model[1].W, model[2].W, Ξ, model[3].W, x_batch, ẋ_batch, ż_ref_batch, dec_jac_batch, alphas, method, biginds)
    end

    model[3].W[biginds] .= Ξ

    # Compute the average loss for the epoch
    epoch_loss /= num_batches
    
    # Store the epoch loss
    push!(final_loss_array, epoch_loss)

    # Print loss after some iterations
    if epoch % 100 == 0
        println("Epoch $epoch: Loss: $epoch_loss")
        println("Coefficients: $(model[3].W))")
        println()
    end
end

# Plot the initial loss array
display(plot(log.(final_loss_array), label = "Final Optimization Loss"))

In [ ]:
model[3].W[15] = 1
model[3].W[16] = 1
model[3].W[40] = 0.5
model[3].W[42] = 0.5
model[3].W[77] = 1
model[3].W[78] = 1

In [ ]:
function (vector_field)(dz, z)
    fθ(dz, z, model[3].W)
    return dz
end

(vector_field)(dz, z, p, t) = vector_field(dz, z)

In [ ]:
check = model[3].W

In [ ]:
# ----------------------------------------
# Plot Results
# ----------------------------------------

println("Plotting...")

t_step = 0.01
t_span = (0.0,5.0)

for i in 1:1
    idx = rand(1:length(s))

    prob_reference = ODEProblem((dx, t, x, params) -> grad_H_ana!(dx, x, params, t), t_span, t_step, Float64.(x[:, idx]))
    data_reference = integrate(prob_reference, Gauss(1))

    prob_sindy = ODEProblem((dx, t, x, params) -> vector_field(dx, x, params, t), t_span, t_step, Float64.(model[1].W(x[:, idx])))
    data_sindy = integrate(prob_sindy, Gauss(1))

    xsol = hcat([model[2].W(data_sindy.q[i,:]) for i in axes(data_sindy.q, 1)]...)
    
    p1 = plot(xlabel = "Time", ylabel = "q₁")
    scatter!(p1, data_reference.t, data_reference.q[:,1], label = "Data q₁")
    scatter!(p1, data_sindy.t, xsol[1,:], markershape=:xcross, label = "Identified q₁")

    p3 = plot(xlabel = "Time", ylabel = "p₁")
    scatter!(p3, data_reference.t, data_reference.q[:,3], label = "Data p₁")
    scatter!(p3, data_sindy.t, xsol[3,:], markershape=:xcross, label = "Identified p₁")

    plot!(size=(1000,1000))
    display(plot(p1, p3, title="Analytical vs Calculated q₁ & p₁ in a 2D system with Euler"))

    p2 = plot(xlabel = "Time", ylabel = "q₂")
    scatter!(p2, data_reference.t, data_reference.q[:,2], label = "Data q₂")
    scatter!(p2, data_sindy.t, xsol[2,:], markershape=:xcross, label = "Identified q₂")

    p4 = plot(xlabel = "Time", ylabel = "p₂")
    scatter!(p4, data_reference.t, data_reference.q[:,4], label = "Data p₂")
    scatter!(p4, data_sindy.t, xsol[4,:], markershape=:xcross, label = "Identified p₂")

    plot!(size=(1000,1000))
    display(plot(p2, p4, title="Analytical vs Calculated q₂ & p₂ in a 2D system with Euler"))

end

# ------Zygote Code------

In [ ]:
function evaluate_fθ(x_batch, model)
    out = Zygote.Buffer(x_batch)
    f = Zygote.Buffer(x_batch[:,1])
    enc_x_batch = model[1].W(x_batch)
    for i in 1:size(x_batch, 2)
        # Get the encoded derivative: ż
        fθ(f, enc_x_batch[:,i], model[3].W)
        out[:,i] = f[:]
    end
    return copy(out) 
end

In [ ]:
# Needed because Flux.gradient can't handle Flux.jacobian
function batched_jacobian(model_layer, x_batch)
    output_dim = size(model_layer(x_batch[:, 1]))[1]
    batch_size = size(x_batch, 2)
    
    batch_jac = zeros(Float32, output_dim, batch_size, size(x_batch, 1))
    
    for i in 1:batch_size
        x_input = x_batch[:, i]
        jac = Flux.jacobian(model_layer, x_input)[1]
        batch_jac[:, i, :] = jac
    end
    return batch_jac
end

In [ ]:
# Get ż from encoder derivative and ẋ
function enc_ż(enc_jac_batch, ẋ_batch)
    ż_ref = Zygote.Buffer(ẋ_batch)
    for i in 1:size(enc_jac_batch, 2)
        ż_ref[:, i] = (enc_jac_batch[:,i,:] * (ẋ_batch[:,i]))
    end
    return copy(ż_ref)
end

# Get ẋ from decoder derivative and ż
function dec_ẋ(dec_jac_batch, ż)
    dec_mult_ẋ = Zygote.Buffer(ż)
    for i in 1:size(dec_jac_batch, 2)
        dec_mult_ẋ[:, i] = dec_jac_batch[:,i,:] * ż[:,i]
    end
    return copy(dec_mult_ẋ)
end

In [ ]:
function set_coeffs(model_Coeffs, Ξ, biginds)
    coeffs = Zygote.Buffer(model_Coeffs)
    coeffs[.~biginds] = 0
    coeffs[biginds] = Ξ
    return copy(coeffs)
end

In [ ]:
function loss(model, x_batch, ẋ_batch, dec_jac_batch, ż_ref, alphas, method)
    # Compute the reconstruction loss for the entire batch
    L_r = sum(abs2, model[2].W(model[1].W(x_batch)) - x_batch)

    ### Note: grad_fθ, dec_mult_ẋ, and L_c in loss function so model acts on terms in loss function
    ### and gradient can see that and use that for its update calculations

    # encoded gradient from SINDy
    grad_fθ = evaluate_fθ(x_batch, model)

    L_ż = alphas / 10 * sum(abs2, ż_ref - grad_fθ)

    # decoded SINDy gradient ẋ = dx/dz*grad_fθ
    ẋ_SINDy = dec_ẋ(dec_jac_batch, grad_fθ)

    L_ẋ = alphas * sum(abs2, ẋ_SINDy - ẋ_batch)

    # Compute the total loss for the entire batch
    batchLoss = L_r + L_ż + L_ẋ

    # Mean of the coefficients averaged
    coeffs = model[3].W
    L_c = sum(abs, coeffs) / length(coeffs)

    batch_loss_average = batchLoss / size(x_batch, 2) + method.basis_coeff * L_c
    
    return batch_loss_average
end

In [ ]:
using LinearAlgebra
model = set_model(tdata, method.basis)
model[1].W.layers[1].weight .= Matrix(LinearAlgebra.I, 4, 4)
model[2].W.layers[1].weight .= Matrix(LinearAlgebra.I, 4, 4)

# Flux gradient has problem working with the structure data directly
x = Float32.(tdata.x)
ẋ = Float32.(tdata.ẋ)

total_samples = size(x)[2]
num_batches = ceil(Int, total_samples / method.batch_size)

# Coefficients for the loss_kernel terms
alphas = round(sum(abs2, x) / sum(abs2, ẋ), sigdigits = 3)

# Derivatives of the encoder and decoder
enc_jac_batch = batched_jacobian(model[1].W, x)
dec_jac_batch = batched_jacobian(model[2].W, model[1].W(x))

# encoded gradient ż = ẋ*dz/dx
ż_ref = enc_ż(enc_jac_batch, ẋ)

initial_loss_array = Vector{Float32}()

# initial guess
println("Initial Guess...")

# Set up the optimizer's state
opt_state = Flux.setup(Adam(), model)

@show("Coeffs: $(model[3].W)")
@show("initial total loss:", loss(model, x, ẋ, dec_jac_batch, ż_ref, alphas, method))

In [ ]:
# Array to store the losses
Initial_loss_array = Vector{Float32}()

for epoch in 1:1000
    epoch_loss = 0.0
    # Shuffle the data indices for each epoch
    shuffled_indices = shuffle(1:total_samples)
    
    for batch in 1:num_batches
        # Get the indices for the current batch
        batch_start = (batch - 1) * method.batch_size + 1
        batch_end = min(batch * method.batch_size, total_samples)
        batch_indices = shuffled_indices[batch_start:batch_end]

        # Extract the data for the current batch
        x_batch = x[:, batch_indices]
        ẋ_batch = ẋ[:, batch_indices]
        # Derivatives of the encoder and decoder
        enc_jac_batch = batched_jacobian(model[1].W, x_batch)
        dec_jac_batch = batched_jacobian(model[2].W, model[1].W(x_batch))
        # encoded gradient ż = ẋ*dz/dx
        ż_ref = enc_ż(enc_jac_batch, ẋ_batch)

        # Compute gradients using Flux
        gradients = Flux.gradient(model -> loss(model, x_batch, ẋ_batch, dec_jac_batch, ż_ref, alphas, method), model)[1]

        # Update the parameters
        Flux.Optimise.update!(opt_state, model, gradients)

        # Accumulate the loss for the current batch
        epoch_loss += loss(model, x_batch, ẋ_batch, dec_jac_batch, ż_ref, alphas, method)
    end
    # Compute the average loss for the epoch
    epoch_loss /= num_batches

    # Store the epoch loss
    push!(Initial_loss_array, epoch_loss)

    # Print loss after some iterations
    if epoch % 200 == 0
        println("Epoch $epoch: Average Loss: $epoch_loss")
        println("Epoch $epoch: Coefficents: $(model[3].W)")
        println()
    end
end

# Plot the initial loss array
display(plot(log.(Initial_loss_array), label = "Initial Optimization Loss"))

In [ ]:
# Array to store the losses of each SINDy loop
SINDy_loss_array = Vector{Vector{Float32}}()  # Store vectors of losses

# Initialize smallinds before the loop
smallinds = falses(size(model[3].W))

for n in 1:method.nloops
    println("Iteration #$n...")
    println()
    # find coefficients below λ threshold
    smallinds .= abs.(model[3].W) .< method.λ

    biginds = .~smallinds

    # check if there are any small coefficients != 0 left
    all(model[3].W[smallinds] .== 0) && break

    # set all small coefficients to zero
    model[3].W[smallinds] .= 0

    # Set up the optimizer's state
    Ξ = model[3].W[biginds]

    function sparse_loss(enc_paras, dec_paras, Ξ, model_Coeffs, x_batch, ẋ_batch, ż_ref, dec_jac_batch, alphas, method, biginds)
        coeffs = set_coeffs(model_Coeffs, Ξ, biginds)

        enc_x_batch = enc_paras(x_batch)
    
        # Compute the reconstruction loss for the entire batch
        L_r = sum(abs2, dec_paras(enc_x_batch) - x_batch)
    
        # Note: grad_fθ, dec_mult_ẋ, and L_c in loss function so model acts on terms in loss function
        # and gradient can see that and use that for its update calculations
    
        # encoded gradient from SINDy
        grad_fθ = evaluate_fθ(fθ, enc_x_batch, coeffs)
    
        # Difference b/w encoded gradients from SINDy and reference
        L_ż = alphas / 10 * sum(abs2, ż_ref - grad_fθ)
    
        # Decoded SINDy gradient ẋ = dx/dz*grad_fθ
        ẋ_SINDy = dec_ẋ(dec_jac_batch, grad_fθ)
        L_ẋ = alphas * sum(abs2, ẋ_SINDy  - ẋ_batch)
    
        # Compute the total loss for the entire batch
        batchLoss = L_r + L_ż + L_ẋ
    
        # Mean of the coefficients averaged
        L_c = sum(abs, model_Coeffs) / length(model_Coeffs)
    
        batch_loss_average = batchLoss / size(x_batch, 2) + method.basis_coeff * L_c
        
        return batch_loss_average
    end
    
    # Flux gradient has problem working with the structure data directly
    x = Float32.(tdata.x)
    ẋ = Float32.(tdata.ẋ)

    total_samples = size(x)[2]
    num_batches = ceil(Int, total_samples / method.batch_size)

    # Coefficients for the loss_kernel terms
    alphas = round(sum(abs2, x) / sum(abs2, ẋ), sigdigits = 3)

    # Array to store the losses of each epoch
    epoch_loss_array = Vector{Float64}()

    # Set up the optimizer's state
    opt_state = Flux.setup(Adam(), (model[1].W, model[2].W, Ξ))

    for epoch in 1:500
        epoch_loss = 0.0
        # Shuffle the data indices for each epoch
        shuffled_indices = shuffle(1:total_samples)

        for batch in 1:num_batches
            # Get the indices for the current batch
            batch_start = (batch - 1) * method.batch_size + 1
            batch_end = min(batch * method.batch_size, total_samples)
            batch_indices = shuffled_indices[batch_start:batch_end]

            # Extract the data for the current batch
            x_batch = x[:, batch_indices]
            ẋ_batch = ẋ[:, batch_indices]

            # Derivatives of the encoder and decoder
            enc_jac_batch = batched_jacobian(model[1].W, x_batch)
            dec_jac_batch = batched_jacobian(model[2].W, model[1].W(x_batch))
            
            # Get the encoded derivative: ż
            ż_ref_batch = enc_ż(enc_jac_batch, ẋ_batch)

            # Compute gradients using Flux
            gradients = Flux.gradient((enc_paras, dec_paras, Ξ) -> sparse_loss(enc_paras, dec_paras, Ξ, model_Coeffs, x_batch, ẋ_batch, ż_ref, dec_jac_batch, alphas, method, biginds), model[1].W, model[2].W, Ξ)

            # Update the parameters
            Flux.Optimise.update!(opt_state, (model[1].W, model[2].W, Ξ), gradients)

            # Accumulate the loss for the current batch
            epoch_loss += sparse_loss(model[1].W, model[2].W, Ξ, model[3].W, x_batch, ẋ_batch, ż_ref_batch, dec_jac_batch, alphas, method, biginds)
        end

        model[3].W[biginds] .= Ξ

        # Compute the average loss for the epoch
        epoch_loss /= num_batches
        
        # Store the epoch loss
        push!(epoch_loss_array, epoch_loss)

        # Print loss after some iterations
        if epoch % 100 == 0
            println("Epoch $epoch: Loss: $epoch_loss")
            println("Coefficients: $(model[3].W))")
            println()
        end
    end

    # Store the SINDy loop loss
    push!(SINDy_loss_array, epoch_loss_array)
end

# Convert vector of vectors to a single vector
SINDy_loss_array = vcat(SINDy_loss_array...)

# Plot the initial loss array
display(plot(log.(SINDy_loss_array), label = "SINDy Optimization Loss"))